In [1]:
import pydicom
import PIL
from PIL import ImageOps
import numpy as np
import os
from PIL.Image import fromarray
from pydicom.pixel_data_handlers.util import apply_voi_lut
from pydicom.pixel_data_handlers.util import apply_modality_lut
import cv2
from skimage import color

print("done2")

done2


In [2]:
## Access dicom images

def get_names(path):
    names = []
    paths = []
    pIDs = []
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            _, ext = os.path.splitext(filename)
            if ext in ['.dcm']:
                names.append(filename)
                paths.append(root+"/"+filename)
                pIDs.append(root.replace((os.path.dirname(root)+"/"), ''))
#                 print()
#                 print(root)
#                 print(filename)
#                 print(root.replace((os.path.dirname(root)+"/"), ''))
    return (names, paths, pIDs)


## Convert dicom to jpg
def convert_dcm_jpg(name):
    from PIL.Image import fromarray

    im = pydicom.dcmread(name)
    
    # Problem 1: not all dcm files are uncompressed, therefore make sure to decompress all files (harmless operation)
    im.decompress()
        
    elem = im[0x0028, 0x0004]
    
    # Exceptions case 
#     if 'WindowWidth' in im:
#         print('Dataset has windowing')
        
    # Problem 2: something something window values, something something (harmless operation)
    im = apply_voi_lut(im.pixel_array.astype(float), im)
    
#     im = im.pixel_array.astype(float)

    # Change pixelvalues into image values
    rescaled_image = (np.maximum(im,0)/im.max())*255 # float pixels
    final_image = np.uint8(rescaled_image) # integers pixels
    
    # Problem 3: Files can be monochrome1 or monochrome2 encoded, if monochrome1 then all blacks are white and vice versa
    if 'MONOCHROME1' in elem.value:
        final_image = np.invert(final_image)
    
    # Problem 4: Sometimes a file isn't monochrome encoded but in RGB color. Convert it to grayscale.
    if 'RGB' in elem.value:
        R, G, B = final_image[:,:,0], final_image[:,:,1], final_image[:,:,2]
        final_image = 0.2989 * R + 0.5870 * G + 0.1140 * B
        final_image = np.uint8(final_image) # integers pixels
#         print(final_image)
    
    # Problem 5: This checks if there are multiple (in this case 2) frames, so you only use the first image
    if final_image.shape[0] == 2:
        final_image = final_image[0]
        
    # CLAHE operation: enhances the contrast of everyimage, standardizing the lightness values of every image (source needed)    
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    final_image = clahe.apply(final_image)
    
    
    print(final_image.shape)
    
    # Problem 6: Sometimes the image is in the wrong dimensions, transposing and then rotating the image fixes that.
    # This problem might be solved by having solved problem 5.
    final_image = np.transpose(final_image)
    final_image = PIL.Image.fromarray(final_image)
    final_image = final_image.rotate(270, PIL.Image.NEAREST, expand = 1)
    final_image = ImageOps.mirror(final_image)
    return final_image



In [82]:
from PIL.Image import fromarray

path = "/Users/jurreboellen/Downloads/GLORIA films/GLORIA ready for reading-3-copie"
names, paths, pIDs = get_names(path)
for i in range(1):
    print(names[i])
    image = convert_dcm_jpg(paths[i])
#     image = dcm_png(paths[i])
#     ds = pydicom.dcmread(name)
#     image = fromarray(ds.pixel_array)
#     width, height = image.size
#     wid = int(width / 2)
#     img1 = image.crop((0, 0, wid, height))
#     img2 = image.crop((wid, 0, width, height))
    image.show()
#     cv2.imshow("test",image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
#     img1.show()
#     img2.show()    
#     image.save(name+'.png')



106007-B-foot-left.dcm
Dataset has windowing
(1764, 995)


In [3]:
def click_event(event, x, y, flags, params):
    global mouseX, mouseY
    # checking for left mouse clicks
    if event == cv2.EVENT_LBUTTONDOWN:
 
        # displaying the coordinates
        # on the Shell
        print(x, ' ', y)
 
        # displaying the coordinates
        # on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(open_cv_image, ".", (x,y), font,
                    1, (255, 0, 0), 2)
        cv2.imshow('image', open_cv_image)
        mouseX, mouseY = x, y

In [5]:
from pathlib import Path

for folderNo in range(1,13):
#     folderNo = "1"
    path = "/Users/jurreboellen/Downloads/GLORIA films/GLORIA ready for reading-" + str(folderNo) + "-copie"
    names, paths, pIDs = get_names(path)
    for i in range(len(names)):
        names[i] = names[i].lower()
        names[i] = names[i].replace("_", "-")
        names[i] = names[i].replace(" ", "-")
        nameID = names[i]

        patientID = pIDs[i]
        print(patientID)

        # Problem 7: Differently ordered file names
        if nameID[:2] == "im":
            # Use BodyPartExamined in order to create functional nameID
            ds = pydicom.dcmread(paths[i])
            nameID = patientID + "-" + ds[0x0018, 0x0015].value
            nameID = nameID.lower()
            nameID = nameID.replace("_", "-")
            nameID = nameID.replace(" ", "-")
            print(nameID)


        basePath = "/Users/jurreboellen/MCL werk documenten/clean GLORIA " + str(folderNo) + "/"
        storeDir = basePath

        # Convert dcm to png
        image = convert_dcm_jpg(paths[i])    

        #Check which visit
        if "-a-" in nameID:
            storeDir = basePath + patientID + "_A/"
        elif "-b-" in nameID:
            storeDir = basePath + patientID + "_B/"


        #Check for hand
        if "hand" in nameID or "hands" in nameID:
            storeDir = storeDir + "H"

        #Check for foot
        if "foot" in nameID or "feet" in nameID or "voet" in nameID:
            storeDir = storeDir + "F"

        # Check for plurals
        if "both" in nameID or "hands" in nameID or "feet" in nameID or "bdz" in nameID or "left&right" in nameID:
            open_cv_image = np.array(image) 
            cv2.imshow('image', open_cv_image)

            # setting mouse handler for the image
            # and calling the click_event() function
            cv2.setMouseCallback('image', click_event)

            # wait for keypress to continue
            cv2.waitKey(0)
            # close the window
            cv2.destroyAllWindows()
            cv2.waitKey(1) 
            cv2.destroyWindow('image')

            width, height = image.size
    #         wid = int(width / 2)
            imgL = image.crop((0, 0, mouseX, height)) #Left limb
            imgR = image.crop((mouseX, 0, width, height)) #Right limb

            # Store R
            storeDir = storeDir + "R"
            Path(storeDir).mkdir(parents=True, exist_ok=False)
            imgR.save(storeDir + "/" + names[i] + ".png")

            # Store L
            storeDir = storeDir[:-1] + "L"
            Path(storeDir).mkdir(parents=True, exist_ok=False)
            imgL.save(storeDir + "/" + names[i] + ".png")
    #         print(storeDir)
            continue 

        # Check for single left or right
        if "left" in nameID or "links" in nameID:
            storeDir = storeDir + "L"
        else:
            storeDir = storeDir + "R"

        # Check for trash
        if "spine" in nameID or "erected" in nameID:
            storeDir = basePath + "Trash"


        #final storing operation
        #TODO: store file, only dir is created now
        Path(storeDir).mkdir(parents=True, exist_ok=True)
        image.save(storeDir + "/" + names[i] + ".png")
    #     print(storeDir)


    

201001
(1965, 1019)
201001
(1641, 1066)
201001
(1769, 1129)
201001
(1627, 1083)
201001
(1866, 1338)
201001
(1817, 1465)
201001
(1914, 1051)
201001
(1737, 1193)
201002
(1706, 1129)
201002
(1689, 1179)
201002
(1644, 1145)
201002
(1403, 985)
201002
(1708, 1103)
201002
(1392, 811)
201002
(1410, 810)
201002
(1403, 921)
Anonymized - 302001


/Users/jurreboellen/opt/anaconda3/lib/python3.7/site-packages/pydicom/pixel_data_handlers/util.py:424: UserWarning: Applying a VOI LUT on a float input array may give incorrect results
  "Applying a VOI LUT on a float input array may give "


(2828, 2320)
Anonymized - 302001
(2517, 1920)
1010   1764
Anonymized - 302001
(2828, 2320)
Anonymized - 302001
(2920, 2320)
1134   1963
Anonymized - 302001
(2320, 2920)
1507   1375
Anonymized - 302001
(2122, 2719)
1429   892
101035
(2500, 1846)
101035
(2826, 1520)
101035
(2500, 1624)
101035
(2432, 1400)
101035
(2500, 1418)
101035
(2570, 1606)
101035
(2672, 1418)
101035
(2792, 1504)
101003
(1992, 1244)
101003
(1853, 971)
101003
(2395, 1244)
101003
(2133, 1103)
101003
(2057, 1410)
101003
(1928, 1244)
101003
(1894, 928)
101003
(1979, 1103)
101026
(1971, 1402)
101026
(1877, 1244)
101026
(1762, 1244)
101026
(1956, 1244)
101026
(1863, 1244)
101026
(1971, 1482)
101026
(1892, 1266)
101026
(1899, 1287)
101033-only A
(2187, 1294)
101033-only A
(1690, 1244)
101033-only A
(2201, 1244)
101033-only A
(1798, 1244)
101039
(2792, 1298)
101039
(2552, 1554)
101039
(2414, 1128)
101039
(2484, 1332)
101039
(2414, 1674)
101039
(2534, 1400)
101039
(2484, 2016)
101039
(2774, 1418)
101006
(2122, 1359)
101006
(2

106003
(1560, 968)
106003
(1468, 961)
106003
(1512, 912)
106003
(1376, 824)
106003
(1483, 814)
106003
(1666, 1201)
106003
(1606, 788)
106004
(1866, 1056)
106004
(1324, 798)
106004
(1983, 1158)
106004
(1730, 1006)
106004
(1982, 1192)
106004
(1746, 1124)
106004
(1367, 814)
106004
(1524, 916)
106005
(1950, 1322)
106005
(1796, 1058)
106005
(1674, 1101)
106005
(1656, 1017)
106005
(1700, 1026)
106005
(2024, 1239)
106005
(1647, 1037)
106005
(1943, 1070)
106002
(1824, 1115)
106002
(1820, 1177)
106002
(1711, 908)
106002
(1753, 802)
101005
(1986, 2256)
2112   1781
1128   768
101005
(2570, 1692)
101005
(2534, 1350)
101034
(2102, 1397)
101034
(2078, 1349)
101034
(2062, 1454)
101034
(2046, 1470)
101034
(2158, 1903)
876   665
101034
(1813, 1823)
922   659
101032
(2268, 1268)
101032
(2142, 1325)
101032
(2219, 1496)
101032
(2239, 1279)
101032
(2315, 1494)
101032
(1942, 1654)
832   715
101032
(2150, 1309)
101021
(2191, 1333)
101021
(2102, 1943)
980   712
101021
(1773, 1662)
831   594
101021
(2062, 1060

113009
(1663, 2106)
1157   713
301007
(1134, 1322)
662   665
301007
(1352, 1096)
546   734
301007
(1212, 1191)
577   779
572   780
301007
(1217, 1512)
738   681
738   684
737   682
737   679
736   680
737   682
738   682
113008
(2078, 970)
113008
(1520, 1862)
949   600
113008
(2018, 1144)
301009
(1212, 1366)
656   616
654   616
301009
(1332, 1302)
620   845
301009
(1264, 1168)
546   884
301009
(1150, 1132)
536   679
113001
(2111, 1641)
786   562
113001
(2099, 1588)
787   562
113001
(1806, 1603)
817   682
113001
(1728, 1934)
981   687
113011
(2178, 1719)
809   661
113011
(2043, 1838)
844   830
113010
(2097, 1756)
851   617
113010
(2052, 1624)
810   629
113010
(1897, 1917)
897   749
113010
(1696, 1799)
885   795
115001
(1896, 1506)
749   619
115001
(1920, 1520)
749   738
746   740
304002
(1252, 1976)
977   645
304002
(2304, 2760)
1327   1208
304002
(1469, 1660)
823   977
304002
(2136, 2028)
1074   1414
304005
(1386, 2281)
1110   644
304005
(1091, 1824)
904   537
304005
(1755, 1705)
853  

107014_A
107014-a-hand-rechts-1r
(1786, 1260)
107006_B
107006-b-hand-rechts-1r
(2368, 1656)
107006_B
107006-b-voet-links-1r
(2236, 1382)
107006_B
107006-b-hand-links-1r
(2370, 1721)
107006_B
107006-b-voet-rechts-1r
(2278, 1310)
107004_A
107004-a-voet-links-1r
(1663, 988)
107004_A
107004-a-hand-links-1r
(2046, 1473)
107004_A
107004-a-voet-rechts-1r
(1683, 921)
107004_A
107004-a-hand-rechts-1r
(1687, 1251)
107023_B
107023-b-voet-rechts-1r
(1935, 1174)
107023_B
107023-b-hand-links-1r
(2084, 1826)
107023_B
107023-b-voet-links-1r
(2043, 1082)
107023_B
107023-b-hand-rechts-1r
(2037, 1712)
107021_A
107021-a-voet-rechts-1r
(1729, 1010)
107021_A
107021-a-hand-rechts-1r
(2173, 1338)
107021_A
107021-a-voet-links-1r
(1702, 1112)
107021_A
107021-a-hand-links-1r
(2175, 1441)
107008_B
107008-b-voet-rechts-1r
(1915, 1105)
107008_B
107008-b-hand-links-1r
(1919, 1354)
107008_B
107008-b-voet-links-1r
(1741, 986)
107008_B
107008-b-hand-rechts-1r
(2164, 1526)
112040
(1977, 1148)
112040
(1950, 1088)
112040


104008
(2586, 1470)
104008
(2432, 1504)
104009
(2256, 2570)
1289   921
104009
(2878, 2034)
104009
(2878, 1436)
104009
(2050, 2176)
1136   730
104009
(2740, 1436)
104009
(2878, 1776)
104007
(2090, 2256)
1176   760
104007
(2654, 1606)
104007
(2484, 1572)
104007
(2118, 2740)
1241   884
104007
(2774, 1726)
104007
(2654, 1674)
104013
(2256, 2346)
1300   829
104013
(2118, 2586)
1459   675
104013
(2774, 1794)
104013
(2878, 1726)
104013
(2808, 1914)
104013
(2792, 1828)
104014
(2016, 2432)
1269   817
104014
(2740, 1624)
104014
(2774, 1624)
104015
(2260, 1298)
104015
(2740, 1470)
104015
(1828, 2056)
1041   801
104015
(1846, 2278)
1116   771
104015
(2244, 1298)
104015
(2552, 1162)
104012
(1914, 2210)
1077   755
104012
(2654, 1658)
104012
(2604, 1658)
116012
(1512, 1146)
116012
(1542, 996)
116012
(1532, 986)
116012
(1532, 1156)
116015
(1458, 848)
116015
(1458, 848)
116015
(2690, 1914)
116015
(2844, 1470)
116015
(2774, 1812)
116015
(2878, 1504)
116014
(1660, 934)
116014
(1606, 922)
116014
(1648, 10

698   537
108028
(1846, 1548)
783   568
108028
(1681, 1308)
626   534
108021
(1654, 2156)
1038   856
108021
(1470, 1337)
682   477
108021
(1284, 1035)
493   361
108021
(1641, 1614)
748   570
108026
(1337, 1257)
612   608
108026
(1420, 1340)
695   525
108027
(1310, 1212)
641   604
108027
(1461, 1326)
692   650
108018
(1386, 1115)
540   755
108018
(1260, 1428)
674   502
108020
(1495, 1220)
580   620
108020
(1525, 1148)
560   585
108011
(1445, 1317)
710   463
108011
(1685, 1678)
783   551
108011
(1800, 1904)
936   765
108011
(1473, 1499)
743   712
740   718
108042
(1602, 2001)
947   698
108042
(1570, 1375)
650   1277
117005
(1935, 1410)
117005
(2266, 1309)
117005
(2230, 1381)
117005
(2035, 1474)
117005
(2438, 1906)
117005
(1971, 1316)
111008
(2009, 1109)
111008
(1562, 929)
111008
(1632, 921)
111008
(1652, 920)
111008
(1869, 1107)
111008
(2055, 1121)
111008
(1927, 1007)
111008
(1656, 895)
111009
(1935, 895)
111009
(1532, 770)
111009
(1256, 822)
111009
(1586, 793)
111009
(1825, 1107)
111009

1251   854
501018
(1576, 1976)
975   621
501018
(1976, 1576)
783   756
501027
(2436, 2330)
1093   732
501027
(3020, 2520)
1276   1179
501027
(2520, 3032)
1579   1023
501027
(2520, 2368)
1162   776
501029
(3020, 2400)
501029
(3020, 2400)
501044
(2456, 3052)
1557   1060
501044
(1976, 1576)
766   640
501044
(1576, 1976)
1081   275
501044
(2456, 3052)
1469   882
501043
(2646, 2517)
1258   1039
501043
(2048, 2500)
1211   726
501043
(2400, 3020)
1512   945
501043
(2897, 2277)
1141   797
501028
(2050, 1162)
501028
(2058, 1493)
501028
(2069, 1858)
907   536
501010
(2400, 3020)
1737   882
501010
(3020, 2400)
1228   562
501019
(2000, 1676)
836   659
501019
(2500, 2048)
993   1485
501026
(1976, 1576)
780   575
501026
(1576, 1976)
972   871
305004-hand only
(2373, 2373)
1131   1523
305004-hand only
(1576, 1976)
959   847
305005-hand only
(1576, 1976)
971   657
983   663
983   673
305005-hand only
(2373, 2836)
1406   1200
305007
(1576, 1976)
957   972
305007
(2373, 2836)
1335   1242
305007
(1713, 2

In [20]:
path = "/Users/jurreboellen/Downloads/GLORIA films/GLORIA ready for reading-12-copie"
names, paths, pIDs = get_names(path)
print(len(names))

3


In [21]:
print(len(names))
for i in range(len(sorted(names))):
    print(names[i], pIDs[i])

3
501059-B-hand-both.dcm 501059-B
501005-B-foot both.dcm 501005-B
501005-B-hand-both.dcm 501005-B


In [3]:
### Special case for folder filled with .jpg instead of .dcm

from pathlib import Path

def get_names_jpg(path):
    names = []
    paths = []
    pIDs = []
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            _, ext = os.path.splitext(filename)
            if ext in ['.jpg']:
                names.append(filename)
                paths.append(root+"/"+filename)
                pIDs.append(root.replace((os.path.dirname(root)+"/"), ''))
#                 print()
#                 print(root)
#                 print(filename)
#                 print(root.replace((os.path.dirname(root)+"/"), ''))
    return (names, paths, pIDs)

for folderNo in range(1,13):
#     folderNo = "5"
    path = "/Users/jurreboellen/Downloads/GLORIA films/GLORIA ready for reading-" + str(folderNo) + "-copie"
    names, paths, pIDs = get_names_jpg(path)
    for i in range(len(names)):
        names[i] = names[i].lower()
        names[i] = names[i].replace("_", "-")
        names[i] = names[i].replace(" ", "-")
        nameID = names[i]

        patientID = pIDs[i]
        print(patientID)

        # Problem 7: Differently ordered file names
        if nameID[:2] == "im":
            # Use BodyPartExamined in order to create functional nameID
            ds = pydicom.dcmread(paths[i])
            nameID = patientID + "-" + ds[0x0018, 0x0015].value
            nameID = nameID.lower()
            nameID = nameID.replace("_", "-")
            nameID = nameID.replace(" ", "-")
            print(nameID)


        basePath = "/Users/jurreboellen/MCL werk documenten/clean GLORIA " + str(folderNo) + "/"
        storeDir = basePath

        # Convert dcm to png
        image = PIL.Image.open(paths[i])    

        #Check which visit
        if "-a-" in nameID:
            storeDir = basePath + patientID + "_A/"
        elif "-b-" in nameID:
            storeDir = basePath + patientID + "_B/"


        #Check for hand
        if "hand" in nameID or "hands" in nameID:
            storeDir = storeDir + "H"

        #Check for foot
        if "foot" in nameID or "feet" in nameID or "voet" in nameID:
            storeDir = storeDir + "F"

        # Check for plurals
        if "both" in nameID or "hands" in nameID or "feet" in nameID or "bdz" in nameID or "left&right" in nameID:
#             open_cv_image = np.array(image) 
#             cv2.imshow('image', open_cv_image)

#             # setting mouse handler for the image
#             # and calling the click_event() function
#             cv2.setMouseCallback('image', click_event)

#             # wait for keypress to continue
#             cv2.waitKey(0)
#             # close the window
#             cv2.destroyAllWindows()
#             cv2.waitKey(1) 
#             cv2.destroyWindow('image')

            width, height = image.size
            wid = int(width / 2)
            imgL = image.crop((0, 0, wid, height)) #Left limb
            imgR = image.crop((wid, 0, width, height)) #Right limb

            # Store R
            storeDir = storeDir + "R"
            Path(storeDir).mkdir(parents=True, exist_ok=False)
            imgR.save(storeDir + "/" + names[i] + ".png")

            # Store L
            storeDir = storeDir[:-1] + "L"
            Path(storeDir).mkdir(parents=True, exist_ok=False)
            imgL.save(storeDir + "/" + names[i] + ".png")
    #         print(storeDir)
            continue 

        # Check for single left or right
        if "left" in nameID or "links" in nameID:
            storeDir = storeDir + "L"
        else:
            storeDir = storeDir + "R"

        # Check for trash
        if "spine" in nameID:
            storeDir = basePath + "Trash"


        #final storing operation
        #TODO: store file, only dir is created now
        Path(storeDir).mkdir(parents=True, exist_ok=False)
        image.save(storeDir + "/" + names[i] + ".png")
    #     print(storeDir)


    

301002
301002
301002
301002
301007
301007
301007
301007
301009
301009
301009
301009
301005
301005
301005
301005
301004
301004
301004
301004
701016
701016
701016
701020
701020
701020
701020
701020
701020
701042
701042
701042
701026
701026
701010
701010
701017
701017
701017
701017
701017
701044
701044
701032
701032
701032
701004
701004
701004
701004
701003
701003
701003
701003
701003
701003
701002
701002
701002
701002
701002
701005
701005
701005
701005
701041
701041
701041
701040
701040
701040
701040
701040
701025
701025
701022
701022
701014
701014
701014
701014
701014
701014
701013
701013
701013
701013
701031
701031
701009
701009
701009
701009
701036
701036
701007
701007
701007
701007
701008
701008
701008
701008
701011
701011
701027
701027
701045
701045
701045
701021
701021
701019
701019
701028
701028
701043
701043
701035
701035
701034
701034
701034
701034
701012
701012
701015
701015
701023
701023
701024
701038
701038
701039
701039
701001
701001
701001
701037
701037
701037
701018
701018